## Heroes Of Pymoli - Analysis - Wendy Chau

* There is no significant difference between purchasing habits between Males & Females.  Males make up 84.03% of the players and made 83.59% (652 of 780) of the total purchases; on the other hand Females are 14.06% of the population of players and made 14.49% (113 of 780) of the total purchases.  In a similar vein, no specific age group stands out as being more likely to make purchases.

* The player base in this game skews towards young adults, with the age groups 15-19, 20-24, & 25-29 making up 18.58%, 44.79%, & 13.37% of the player base, respectively.  These groups tend to be targeted by marketers (young adults are trendy and have buying power), which is accurately reflected as the top five spenders in this game (Lisosia93, Idastidru52, Chamjask73, Iral74, & Iskadarya95) are 25, 24, 22, 21, & 20 years old, respectively.

* Players tend to buy pricier items.  One anomaly is the item named "Pursuit, Cudgel of Necromancy" - it's very popular (with eight purchases) while being inexpensive at the same time (1.02 versus four-dollar items).  There should be further analysis on that item to see if price or stat adjustment is necessary encourage higher revenues.

----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

In [2]:
total_unique_players = pd.DataFrame({"Total Players" : [len(purchase_data["SN"].unique())]})
total_unique_players

,Total Players
0,576


## Purchasing Analysis (Total)

In [3]:
purchasing_analysis_total = pd.DataFrame(
    {
        "Number of Unique Items" : [len(purchase_data["Item ID"].unique())],
        "Average Price" : [purchase_data["Price"].mean()],
        "Number of Purchases" : [purchase_data["Purchase ID"].count()],
        "Total Revenue" : [purchase_data["Price"].sum()],
        }                            
)

# re-formatting numbers
purchasing_analysis_total["Average Price"] = purchasing_analysis_total["Average Price"].map("${:.2f}".format)
purchasing_analysis_total["Total Revenue"] = purchasing_analysis_total["Total Revenue"].map("${:,.2f}".format)

# re-arranging columns
purchasing_analysis_total = purchasing_analysis_total[["Number of Unique Items", "Average Price", "Number of Purchases","Total Revenue"]]

purchasing_analysis_total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics


In [4]:
gender_demographics =  pd.DataFrame(purchase_data.groupby("Gender")["SN"].nunique())
 
gender_demographics = gender_demographics.rename(columns = {"SN":"Total Count"})    
    
gender_demographics["Percentage of Players"] = gender_demographics["Total Count"] / len(purchase_data["SN"].unique())

# re-formatting and changing order of rows
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{:.2%}".format)

gender_demographics = gender_demographics.reindex(["Male", "Female", "Other / Non-Disclosed"])

gender_demographics

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

In [5]:
# various calculations

purchasing_analysis_gender = pd.DataFrame(
    {
        "Purchase Count": purchase_data["Gender"].value_counts(),
        "Average Purchase Price": purchase_data.groupby('Gender')["Price"].mean(),
        "Total Purchase Value": purchase_data.groupby('Gender')["Price"].sum()
     }
)
purchasing_analysis_gender["Avg Total Purchase per Person"] = purchase_data.groupby('Gender')["Price"].sum() / gender_demographics["Total Count"]

# re-formatting
purchasing_analysis_gender["Average Purchase Price"] = purchasing_analysis_gender["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_gender["Total Purchase Value"] = purchasing_analysis_gender["Total Purchase Value"].map("${:,.2f}".format)
purchasing_analysis_gender["Avg Total Purchase per Person"] = purchasing_analysis_gender["Avg Total Purchase per Person"].map("${:.2f}".format)

purchasing_analysis_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [6]:
# creating bins
bins = [0,9,14,19,24,29,34,39,45]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# looking at ages of unique users only
unique_users_only = purchase_data.drop_duplicates(subset = ["SN"], keep = 'first')

# putting unique users into bins
categorize_by_age = pd.cut(unique_users_only["Age"], bins, labels=group_names)

# counting unique users in each age group
group_by_age = pd.DataFrame(categorize_by_age.value_counts()).reindex(["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])

group_by_age = group_by_age.rename(columns={"Age":"Total Count"})

# calculating percentage using number of unique users
group_by_age["Percentage of Players"] = group_by_age["Total Count"] / len(purchase_data["SN"].unique())

# re-formatting
group_by_age["Percentage of Players"] = group_by_age["Percentage of Players"].map("{:.2%}".format)

group_by_age

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [7]:
# grabbing age data
purchasing_analysis_age = purchase_data.loc[:, ["Age", "Price"]] 

# sorting into age group bins
purchasing_analysis_age["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_names)

# calculations
purchase_by_age = pd.DataFrame(
    {
        "Purchase Count": purchasing_analysis_age["Age Group"].value_counts(),
        "Total Purchase Value": purchasing_analysis_age.groupby("Age Group")["Price"].sum(),
        }
)

purchase_by_age["Average Purchase Price"] = purchase_by_age["Total Purchase Value"] / purchase_by_age["Purchase Count"]

# re-organizing by age group order
purchase_by_age = purchase_by_age.reindex(["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])

# once in same order as previous dataframe, use both to calculate average total purchase per person
purchase_by_age["Avg Total Purchase per Person"] = purchase_by_age["Total Purchase Value"] / group_by_age["Total Count"]

# re-formatting and changing order of columns
purchase_by_age["Total Purchase Value"] = purchase_by_age["Total Purchase Value"].map("${:,.2f}".format)
purchase_by_age["Average Purchase Price"] = purchase_by_age["Average Purchase Price"].map("${:.2f}".format)
purchase_by_age["Avg Total Purchase per Person"] = purchase_by_age["Avg Total Purchase per Person"].map("${:.2f}".format)

purchase_by_age = purchase_by_age[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]

purchase_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [8]:
# grabbing top five users based on dollar amount spent

top_five_spenders = pd.DataFrame(
    {
        "Total Purchase Value": purchase_data.groupby("SN")["Price"].sum(),
        "Purchase Count": purchase_data.groupby("SN")["Price"].count()
        }).sort_values(["Total Purchase Value"], ascending = False).head(5)

# basic calculation
top_five_spenders["Average Purchase Price"] = top_five_spenders["Total Purchase Value"] / top_five_spenders["Purchase Count"]

# re-formatting and changing order of columns
top_five_spenders["Total Purchase Value"] = top_five_spenders["Total Purchase Value"].map("${:.2f}".format)
top_five_spenders["Average Purchase Price"] = top_five_spenders["Average Purchase Price"].map("${:.2f}".format)

top_five_spenders = top_five_spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

top_five_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [9]:
# isolating item data
item_data = purchase_data.loc[:, ["Item ID", "Item Name", "Price"]]

most_popular_items = pd.DataFrame(item_data.groupby("Item ID")["Item ID"].count())

most_popular_items = most_popular_items.rename(columns = {"Item ID":"Purchase Count"}).sort_values(["Purchase Count"], ascending = False)

# merge as a means of looking up corresponding item name & price, cleaning up data post-merge
most_popular_items = pd.merge(most_popular_items, item_data, on = "Item ID", how = "left").drop_duplicates(subset = ["Item ID"], keep = 'first').reset_index(drop = True)

most_popular_items["Total Purchase Value"] = most_popular_items["Purchase Count"] * most_popular_items["Price"]

# re-formatting and changing order of columns
most_popular_items = most_popular_items.rename(columns={"Price":"Item Price"})

most_popular_items = most_popular_items[["Item ID", "Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]

# created separate df to re-format to retain integer/float in original df (when mapping $, changes to string)
popular_items_reformat = most_popular_items.loc[:, ["Item ID", "Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]

popular_items_reformat["Item Price"] = popular_items_reformat["Item Price"].map("${:.2f}".format)
popular_items_reformat["Total Purchase Value"] = popular_items_reformat["Total Purchase Value"].map("${:.2f}".format)

popular_items_reformat = popular_items_reformat.set_index("Item ID")

popular_items_reformat.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [10]:
# sorting by highest total purchase value

most_profitable_items = pd.DataFrame(most_popular_items).sort_values(["Total Purchase Value"], ascending = False)

# re-formatting
most_profitable_items["Item Price"] = most_profitable_items["Item Price"].map("${:.2f}".format)
most_profitable_items["Total Purchase Value"] = most_profitable_items["Total Purchase Value"].map("${:.2f}".format)

most_profitable_items = most_profitable_items.set_index("Item ID")

most_profitable_items.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


### please see written analysis at the top of this jupyter notebook -- thanks